In [ ]:
# See deep_learning_with_keras_python.ipynb for explanations of everything!

# Import packages we'll be using
import numpy as np
import keras

np.random.seed(2018)

# Load iris data
iris = np.load("data/iris.npy")
iris

In [ ]:
# Prepare data
np.random.shuffle(iris)

iris_labels = iris[:, 4]

from keras.utils import to_categorical
iris_onehot = to_categorical(iris_labels)
iris_onehot

In [ ]:
# Build the model
from keras.models import Sequential

model = Sequential()

from keras.layers import Dense

model.add(Dense(15, activation="sigmoid", input_shape=(4,)))
model.add(Dense(3, activation="softmax"))

model.summary()

In [ ]:
# Compile and fit
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

model.fit(iris[:, :4], iris_onehot, epochs=50, batch_size=20, validation_split=0.2)

In [ ]:
# Load walking data
walking = np.load("data/walking_data.npy")
walking.shape

In [ ]:
# Set up a quick plotting function
import matplotlib.pyplot as plt
%matplotlib inline

def plot_series(series):
    # x-channel
    plt.plot(series[:, 0], color="red")
    # y-channel
    plt.plot(series[:, 1], color="green")
    # z-channel
    plt.plot(series[:, 2], color="blue")

plot_series(walking[100, :, :])

In [ ]:
# Import labels (they're stored separately!)
walking_labels = np.load("data/walking_labels.npy")
set(walking_labels)

In [ ]:
# Prepare partitions for training/testing
m = walking.shape[0]
indices = [x for x in range(m)]
np.random.shuffle(indices)

train_indices = indices[:int(m*0.6)]
val_indices = indices[int(m*0.6):int(m*0.8)]
test_indices = indices[int(m*0.8):]

X_train = walking[train_indices, :, :]
X_val = walking[val_indices, :, :]
X_test = walking[test_indices, :, :]

y_train = to_categorical(walking_labels[train_indices])
y_val = to_categorical(walking_labels[val_indices])
y_test = to_categorical(walking_labels[test_indices])

In [ ]:
# Create the model
model = Sequential()

from keras.layers import Conv1D, MaxPooling1D, Flatten

model.add(Conv1D(filters=30, kernel_size=40, strides=2, activation="relu", input_shape=(260, 3)))

model.add(MaxPooling1D(pool_size=2))


model.add(Conv1D(filters=30, kernel_size=10, activation="relu"))
model.add(MaxPooling1D(pool_size=2))

# Check model output
model.output_shape

In [ ]:
model.add(Flatten())
model.output_shape

In [ ]:
model.add(Dense(100, activation="sigmoid"))
model.add(Dense(15, activation="softmax"))

# See what we've done!
model.summary()

In [ ]:
# Compile and fit
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=100, validation_data=(X_val, y_val))

In [ ]:
# Prediction and reporting
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict_classes(X_test)

print(classification_report(np.argmax(y_test, axis=1), y_pred))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

In [ ]:
# Looking inside the neurons
def plot_filter(model, layer, k):
    weights = model.layers[layer].get_weights()[0][:, :, k]
    plot_series(weights)
    
plot_filter(model, 0, 4)

In [ ]:
# Autocorrelation of learned weights
def plot_filter_corr(model, layer, k):
    weights = model.layers[layer].get_weights()[0][:, :, k]
    corrs = np.apply_along_axis(lambda y: np.correlate(y, y, mode="full"), 0, weights)
    plot_series(corrs[corrs.shape[0]//2:, :])
    
plot_filter_corr(model, 0, 4)